In [17]:
from transformers.trainer_utils import set_seed
from datasets import load_dataset
import warnings
from transformers import AutoTokenizer, BatchEncoding, AutoModel, EvalPrediction, TrainingArguments, Trainer
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from transformers.utils import ModelOutput
from scipy.stats import spearmanr
from datasets import Dataset
from torch.utils.data import DataLoader
from pprint import pprint
import csv
import random
from typing import Iterator
import numpy as np
import faiss

In [13]:
warnings.simplefilter('ignore')

In [14]:
# 乱数の固定
set_seed(42)

In [15]:
# データのロード
unsup_train_dataset = load_dataset('llm-book/jawiki-sentences', split="train")

print(unsup_train_dataset)

Generating train split:   0%|          | 0/24387500 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 24387500
})


---
＜ポイント＞  
・textの列しかもっていない   
・2千万のデータ 

---

In [17]:
# 最初の50件を表示
for i, text in enumerate(unsup_train_dataset[:50]['text']):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

---
＜ポイント＞  
・空白の行があるので、これを前処理で除く必要がある。

---

In [20]:
# 空行を削除
unsup_train_dataset = unsup_train_dataset.filter(lambda example: example['text'].strip() != "")



Filter:   0%|          | 0/23048277 [00:00<?, ? examples/s]

In [21]:
unsup_train_dataset[41:45]

{'text': ['また、主にマイクロソフト系では整数の十六進表記に &h を用い、&h0F (十進で15)のように表現する。',
  'SGML、XML、HTMLでは、アンパサンドを使ってSGML実体を参照する。',
  '言語(げんご)は、狭義には「声による記号の体系」をいう。',
  '広辞苑や大辞泉には次のように解説されている。']}

In [23]:
# 100万データをランダムサンプリング
unsup_train_dataset = unsup_train_dataset.shuffle().select(range(1000000))

# 参照によるマッピングではなく、シャッフルしてサンプリングされたデータセットをディスクに書き込む
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [31]:
# dataの確認
print(unsup_train_dataset)

print()
print("="*50)
print()

# シャッフルされているかの確認
for i, text in enumerate(unsup_train_dataset[:10]['text']):
    print(i, text)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


0 その学校は基本モード手法からアヴァンギャルド系のジャズ学校でリディアン・クロマティック・コンセプトの大家ジョージ・ラッセル(英語版)も時折訪れて指導をしていた。
1 延暦5年(786年)内舎人となったが、3年後の延暦8年(789年)に史都蒙の予言通り、32歳で病により自邸で没した。
2 「あ号作戦後の兵装増備状況調査」によるとあ号作戦(1944年6月)時点での各艦の対空機銃は以下の通りとされている。
3 一方、5月に領土の返還を求めるフランスがタイ領を攻撃し、国際社会への復帰を優先せざるを得ないタイは、1941年に併合した領土の引き渡しに応じ、ナコーン・チャンパーサック県(英語版)(チャンパーサック州)、ピブーンソンクラーム県(英語版)(シェムリアップ州)、プレアタボン県(英語版)(バタンバン州)の3県がフランスに返還された。
4 およそ140万台が生産された。
5 また、海兵寮同期であった山本とは当時からあまりウマが合わず、4歳年長であった日高がしばしば山本を軽んじる態度を見せたためともいわれている。
6 これら初期の探検家や入植者の中で最も有名な者がダニエル・ブーンであり、伝統的にケンタッキー州創設者の一人と考えられている。
7 全米科学アカデミー会員。
8 他の主力製品はガラス繊維を用いた繊維強化プラスチック (FRP) などの複合材料である。
9 また、このうち三浦、Nanami、有末、トリンドルは『めざましテレビ』(フジテレビ)の『MOTTOいまドキ!』にも出演していた。


In [32]:
# 検証データとテストデータの作成(JSTS)

valid_dataset = load_dataset('llm-book/JGLUE', name='JSTS', split='train')
test_dataset = load_dataset('llm-book/JGLUE', name='JSTS', split='validation')

In [34]:
# tokenizer and collete function

# Tokenizer
base_model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/vocab.txt
loading file spiece.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/tokenizer_config.json
loading file tokenizer.json from cache at None


In [66]:
# collate

def unsup_train_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """教師なしSimCSEの訓練セットのミニバッチを作成"""
    # ミニバッチに含まれる文にトークナイザを適用する
    tokenized_texts = tokenizer(
        [example["text"] for example in examples],
        padding=True,
        truncation=True,
        max_length=32,
        return_tensors="pt",
    )

    # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文）に対してi列目の事例（文）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    return {
        "tokenized_texts_1": tokenized_texts,
        "tokenized_texts_2": tokenized_texts,
        "labels": labels,
    }

In [61]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
    """SimCSEの検証・テストセットのミニバッチを作成"""
    # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれに
    # トークナイザを適用する
    tokenized_texts_1 = tokenizer(
        [example["sentence1"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized_texts_2 = tokenizer(
        [example["sentence2"] for example in examples],
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
    # 行列のi行目の事例（文1）に対して
    # i列目の事例（文2）との組が正例ペアとなる
    labels = torch.arange(len(examples))

    # データセットに付与された類似度スコアのTensorを作成する
    label_scores = torch.tensor(
        [example["label"] for example in examples]
    )

    return {
        "tokenized_texts_1": tokenized_texts_1,
        "tokenized_texts_2": tokenized_texts_2,
        "labels": labels,
        "label_scores": label_scores,
    }

In [43]:
valid_dataset[:2]

{'sentence_pair_id': ['0', '1'],
 'yjcaptions_id': ['10005_480798-10996-92616', '100124-104404-104405'],
 'sentence1': ['川べりでサーフボードを持った人たちがいます。', '二人の男性がジャンボジェット機を見ています。'],
 'sentence2': ['トイレの壁に黒いタオルがかけられています。', '2人の男性が、白い飛行機を眺めています。'],
 'label': [0.0, 3.799999952316284]}

In [98]:
class SimCSEModel(nn.Module):
    def __init__(self, base_model_name: str, mlp_only_train: bool = False, temperature: float = 0.05):
        super().__init__()

        self.encoder = AutoModel.from_pretrained(base_model_name)
        self.hidden_size = self.encoder.config.hidden_size
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        self.actiovation = nn.Tanh()

        self.mlp_only_train = mlp_only_train
        self.temperature = temperature

    def encode_text(self, tokenized_texts: BatchEncoding) -> Tensor:
        """エンコーダーを使って文をベクトル変換"""

        # トークン化されたテキストをエンコードする
        encoded_texts = self.encoder(**tokenized_texts)
        
        # 最終層の[CLS]トークンのベクトルを取り出す ->  文ベクトルのスコア
        encoded_texts = encoded_texts.last_hidden_state[:, 0]

        # mlp_only_trainがFlseや推論の場合はヘッドにベクトルを渡さずに返す。
        if self.mlp_only_train and not self.training:
            return encoded_texts


        # MLP層による変換
        encoded_texts = self.dense(encoded_texts)
        encoded_texts = self.actiovation(encoded_texts)

        return encoded_texts


    def forward(
        self, 
        tokenized_texts_1: BatchEncoding,
        tokenized_texts_2: BatchEncoding,
        labels : Tensor,
        label_scores: Tensor | None = None,
    ) -> ModelOutput:
        
        # 各トークン化されたテキストを文ベクトルに変換
        encoded_texts_1 = self.encode_text(tokenized_texts_1)
        encoded_texts_2 = self.encode_text(tokenized_texts_2)

        # コサイン類似度を算出
        #  tokenized_texts_2.unsqueeze(0)はブロードキャストを利用？？
        # output-> (batch_size, batch_sizeの大きさのscores): 類似度行列
        sim_matrix = F.cosine_similarity(encoded_texts_1.unsqueeze(1),encoded_texts_2.unsqueeze(0),dim=2,)
        
        loss = F.cross_entropy(sim_matrix / self.temperature, labels)

        # positive score
        positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
        positive_scores = torch.masked_select(sim_matrix, positive_mask)

        
        return ModelOutput(loss=loss, scores=positive_scores)

In [99]:
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/config.json
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-v3",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c

In [100]:
# モデルの訓練と評価をするTrainerを準備
# 今回評価指標として、スペアマンの順位相関係数を使用

# 評価結果の計算関数
def compute_metrics(p: EvalPrediction) -> dict[str, float]:
    """モデルの予想スコアと評価用スコアのスピアマン順位相関係数を計算"""

    scores = p.predictions
    labels, label_scores = p.label_ids

    spearman = spearmanr(scores, label_scores).statistic

    return {'spearman': spearman}

In [101]:
unsup_training_args = TrainingArguments(
    output_dir="outputs_unsup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=64,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=64,  # 評価時のバッチサイズ
    learning_rate=3e-5,  # 学習率
    num_train_epochs=1,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to='all'
)

PyTorch: setting up devices


In [102]:
# 訓練セットと検証セットで異なるcollate関数を使用するのでカスタムtrainerを作成

class SimCSETrainer(Trainer):
    """SimCSEの訓練に使用するTrainer"""

    def get_eval_dataloader(self, eval_dataset: Dataset | None = None) -> DataLoader:

        if eval_dataset is None:
            eval_dataset = self.eval_dataset

        return DataLoader(
            eval_dataset,
            batch_size=64,
            collate_fn=eval_collate_fn, # eval_datasetのcollate関数はeval_collate_fnで固定 (教師あり、教師なしで同じものを使用)
            pin_memory=True
        )




In [103]:
# 教師なしSimCSEのTrainerを初期化する
unsup_trainer = SimCSETrainer(model=unsup_model,
                              args=unsup_training_args,
                              data_collator=unsup_train_collate_fn,
                              train_dataset=unsup_train_dataset,
                              eval_dataset=valid_dataset,
                              compute_metrics=compute_metrics
                             )

Using auto half precision backend


In [104]:
# 訓練の実行
unsup_trainer.train()

***** Running training *****
  Num examples = 1,000,000
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 15,625
  Number of trainable parameters = 111,797,760


Step,Training Loss,Validation Loss,Spearman
250,0.001900,2.516721,0.716522
500,0.000300,2.461056,0.735719
750,0.000400,2.478571,0.733308
1000,0.000200,2.458963,0.739426
1250,0.000300,2.500067,0.729833
1500,0.000200,2.474350,0.726624
1750,0.000100,2.490425,0.726557
2000,0.000000,2.448895,0.733077
2250,0.000300,2.381986,0.734255
2500,0.000300,2.427408,0.728923


***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-250
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_unsup_simcse/checkpoint-250] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-750
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64
Saving model checkpoint to outputs_unsup_simcse/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_unsup_simcse/checkpoint-500] due to args.save_total

TrainOutput(global_step=15625, training_loss=0.00012896459624171257, metrics={'train_runtime': 2971.4218, 'train_samples_per_second': 336.539, 'train_steps_per_second': 5.258, 'total_flos': 0.0, 'train_loss': 0.00012896459624171257, 'epoch': 1.0})

In [105]:
# モデルの評価
unsup_trainer.evaluate(valid_dataset)

***** Running Evaluation *****
  Num examples = 12451
  Batch size = 64


{'eval_loss': 2.2688992023468018,
 'eval_spearman': 0.7576461344472822,
 'eval_runtime': 9.7325,
 'eval_samples_per_second': 1279.319,
 'eval_steps_per_second': 20.036,
 'epoch': 1.0}

In [106]:
unsup_trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 1457
  Batch size = 64


{'eval_loss': 2.139267683029175,
 'eval_spearman': 0.7969628926980149,
 'eval_runtime': 1.3159,
 'eval_samples_per_second': 1107.189,
 'eval_steps_per_second': 17.478,
 'epoch': 1.0}

In [107]:
encoder_path = "outputs_unsup_simcse/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

Configuration saved in outputs_unsup_simcse/encoder/config.json
Model weights saved in outputs_unsup_simcse/encoder/model.safetensors
tokenizer config file saved in outputs_unsup_simcse/encoder/tokenizer_config.json
Special tokens file saved in outputs_unsup_simcse/encoder/special_tokens_map.json


('outputs_unsup_simcse/encoder/tokenizer_config.json',
 'outputs_unsup_simcse/encoder/special_tokens_map.json',
 'outputs_unsup_simcse/encoder/vocab.txt',
 'outputs_unsup_simcse/encoder/added_tokens.json')

# 教師ありSimCSEの実装

In [24]:
set_seed(42)

# データセットのロード
jsnli_dataset = load_dataset("llm-book/jsnli", split="train")

In [25]:
# dataセットの確認
print(jsnli_dataset)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 533005
})


In [26]:
pprint(jsnli_dataset[0])
pprint(jsnli_dataset[1])
pprint(jsnli_dataset[2])
pprint(jsnli_dataset[3])

{'hypothesis': '男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。',
 'label': 'neutral',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'hypothesis': '女性 が 畑 で 踊って い ます 。',
 'label': 'contradiction',
 'premise': '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。'}
{'hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'label': 'contradiction',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}
{'hypothesis': '海 に 転がる 男 。', 'label': 'entailment', 'premise': '海 の 波 に 倒れる 男'}


In [30]:
# 訓練セットの作成

premise2hypotheses = {}

premises = jsnli_dataset['premise']
hypotheses = jsnli_dataset['hypothesis']
labels = jsnli_dataset['label']

for premises, hypotheses, label in zip(premises, hypotheses, labels):
    
    # 前提文の登録がないなら作成
    if premises not in premise2hypotheses:
        premise2hypotheses[premises] = {
            'entailment': [],
            'neutral': [],
            'contradiction': []
        }

    # 各対応するラベルリストに仮設を格納する
    premise2hypotheses[premises][label].append(hypotheses)

In [34]:
premise2hypotheses

{'ガレージ で 、 壁 に ナイフ を 投げる 男 。': {'entailment': ['ガレージ に 男 が い ます 。'],
  'neutral': ['男 は 魔法 の ショー の ため に ナイフ を 投げる 行為 を 練習 して い ます 。'],
  'contradiction': ['男 が 台所 の テーブル で 本 を 読んで い ます 。']},
 '茶色 の ドレス を 着た 女性 が ベンチ に 座って い ます 。': {'entailment': [],
  'neutral': ['女性 が バス停 の ベンチ に 座って い ます 。'],
  'contradiction': ['女性 が 畑 で 踊って い ます 。']},
 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。': {'entailment': ['人 は 椅子 に 座って い ます 。'],
  'neutral': ['若い 男 が ラップ トップ で ポルノ を 探して い ます 。'],
  'contradiction': ['黒人 は デスクトップ コンピューター を 使用 し ます 。']},
 '海 の 波 に 倒れる 男': {'entailment': ['海 に 転がる 男 。'],
  'neutral': ['水 に 倒れた 男 。'],
  'contradiction': ['海 の 波 で 転倒 する 女性']},
 '２ つ の 異なる チーム が ラグビー を して い ます 。': {'entailment': ['一緒に スポーツ を する ２ つ の チーム'],
  'neutral': ['ラグビー で 対戦 する ２ つ の チーム'],
  'contradiction': ['サッカー を する ２ つ の チーム']},
 '空き地 で 男 が 男 と 話し ます 。': {'entailment': ['二 人 が 話して いる 。'],
  'neutral': ['二 人 は 外 に い ます 。'],
  'contradiction': ['２ 人 が レストラン に 座って い ます 。']},
 'セラミック ポット を 作る 膝 の 上 に 小

In [67]:
# 訓練セットを作成するため、generatorを作成
def generate_sup_train_example() -> Iterator[dict[str, str]]:
    """教師ありSimCSEの訓練セットの事例を生成"""

    # JSNLIのデータから (前提文,「含意」ラベルの仮説文,「矛盾」ラベルの仮説文)
    # の三つ組を生成する
    for premise, hypotheses in premise2hypotheses.items():
        # 矛盾ラベルが０の場合、スキップ
        if len(hypotheses['contradiction']) == 0:
            continue

        # entailmentの仮設文1つにつき、contradictionを１つランダムに関連付ける
        # 1つのpremiseで複数のentailmentやcontradictionをもつ場合があるため
        for entailment_hypothesis in hypotheses['entailment']:
            contradiction_hypothesis = random.choice(hypotheses['contradiction'])


            # dictのジェネレーターとして生成する
            yield {
                "premise": premise,
                "entailment_hypothesis": entailment_hypothesis,
                "contradiction_hypothesis": contradiction_hypothesis
            }
            

# 定義したジェネレータ関数を用いて、教師ありSimCSEの訓練セットを構築する
sup_train_dataset = Dataset.from_generator(generate_sup_train_example)

Generating train split: 0 examples [00:00, ? examples/s]

In [68]:
print(sup_train_dataset)

Dataset({
    features: ['premise', 'entailment_hypothesis', 'contradiction_hypothesis'],
    num_rows: 173438
})


In [69]:
pprint(sup_train_dataset[0])
pprint(sup_train_dataset[1])

{'contradiction_hypothesis': '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 'entailment_hypothesis': 'ガレージ に 男 が い ます 。',
 'premise': 'ガレージ で 、 壁 に ナイフ を 投げる 男 。'}
{'contradiction_hypothesis': '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 'entailment_hypothesis': '人 は 椅子 に 座って い ます 。',
 'premise': 'ラップ トップ コンピューター を 使用 して 机 に 座って いる 若い 白人 男 。'}


In [96]:
# collate関数

def sup_train_collate_fn(examples: list[dict]) -> dict[str, BatchEncoding | Tensor]:
    "訓練データセットのミニバッチ処理"
    premises = []
    hypothesis = []

    # [x1のentail, x1のcontra, x2のentail, x2のcontra...]のようなhypothesisリスト作成
    for example in examples:
        premises.append(example['premise'])

        entailment_hypothesis = example['entailment_hypothesis']
        contradiction_hypothesis = example['contradiction_hypothesis']

        hypothesis.extend([entailment_hypothesis, contradiction_hypothesis])

    # tokenizerでベクトル化
    toknized_premise = tokenizer(premises,
                                 padding=True,
                                 truncation=True,
                                 max_length=32,
                                 return_tensors='pt'
                                )

    toknized_hypotheses = tokenizer(hypothesis,
                                    padding=True,
                                     truncation=True,
                                     max_length=32,
                                     return_tensors='pt'
                                    )

    # 正例の位置をlabelとしてリスト化する
    # 行列のi行目の事例（前提文）に対して
    # 2*i列目の要素（仮説文）が正例ペアとなる
    labels = torch.arange(0, 2*len(premises), 2)

    return {
        "tokenized_texts_1": toknized_premise,
        "tokenized_texts_2": toknized_hypotheses,
        "labels":labels
    }


In [97]:
premises = []
hypothesis = []
for idx, example in enumerate(sup_train_dataset):
    premises.append(example['premise'])

    entailment_hypothesis = example['entailment_hypothesis']
    contradiction_hypothesis = example['contradiction_hypothesis']

    hypothesis.extend([entailment_hypothesis, contradiction_hypothesis])
    if idx == 2:
        break

hypothesis

['ガレージ に 男 が い ます 。',
 '男 が 台所 の テーブル で 本 を 読んで い ます 。',
 '人 は 椅子 に 座って い ます 。',
 '黒人 は デスクトップ コンピューター を 使用 し ます 。',
 '海 に 転がる 男 。',
 '海 の 波 で 転倒 する 女性']

In [98]:
# 教師ありSimCSEのモデルを初期化する

# 教師ありは訓練時、推論時にMLP層を使用するので"mlp_only_train=False"を設定
sup_model = SimCSEModel(base_model_name, mlp_only_train=False)

In [99]:
# Trainerの準備
# 教師ありSimCSEの訓練のハイパーパラメータを設定する
sup_training_args = TrainingArguments(
    output_dir="outputs_sup_simcse",  # 結果の保存先フォルダ
    per_device_train_batch_size=128,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=128,  # 評価時のバッチサイズ
    learning_rate=5e-5,  # 学習率
    num_train_epochs=3,  # 訓練エポック数
    evaluation_strategy="steps",  # 検証セットによる評価のタイミング
    eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
    logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
    save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
    metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
)



# ↓教師なしの時との比較用！！
# unsup_training_args = TrainingArguments(
#     output_dir="outputs_unsup_simcse",  # 結果の保存先フォルダ
#     per_device_train_batch_size=64,  # 訓練時のバッチサイズ
#     per_device_eval_batch_size=64,  # 評価時のバッチサイズ
#     learning_rate=3e-5,  # 学習率
#     num_train_epochs=1,  # 訓練エポック数
#     evaluation_strategy="steps",  # 検証セットによる評価のタイミング
#     eval_steps=250,  # 検証セットによる評価を行う訓練ステップ数の間隔
#     logging_steps=250,  # ロギングを行う訓練ステップ数の間隔
#     save_steps=250,  # チェックポイントを保存する訓練ステップ数の間隔
#     save_total_limit=1,  # 保存するチェックポイントの最大数
#     fp16=True,  # 自動混合精度演算の有効化
#     load_best_model_at_end=True,  # 最良のモデルを訓練終了後に読み込むか
#     metric_for_best_model="spearman",  # 最良のモデルを決定する評価指標
#     remove_unused_columns=False,  # データセットの不要フィールドを削除するか
#     report_to='all'
# )

In [100]:
# 教師ありのSimCSE Trainerの初期化
# 

sup_trainer = SimCSETrainer(
    model=sup_model,
    args=sup_training_args,
    data_collator=sup_train_collate_fn,
    train_dataset=sup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [101]:
sup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,1.449800,2.779990,0.790417
500,1.099700,2.700985,0.783842
750,1.019100,2.818404,0.787412
1000,0.968900,2.818200,0.789359
1250,0.932100,2.816230,0.795096
1500,0.835200,2.845381,0.791091
1750,0.756000,2.919118,0.798860
2000,0.752700,2.873110,0.800672
2250,0.745800,2.901493,0.792595
2500,0.729300,2.909121,0.793986


TrainOutput(global_step=4065, training_loss=0.811487272480757, metrics={'train_runtime': 1460.1332, 'train_samples_per_second': 356.347, 'train_steps_per_second': 2.784, 'total_flos': 0.0, 'train_loss': 0.811487272480757, 'epoch': 3.0})

In [102]:
# 検証セットで評価
sup_trainer.evaluate(valid_dataset)

{'eval_loss': 2.8731093406677246,
 'eval_spearman': 0.8006715757961859,
 'eval_runtime': 11.8215,
 'eval_samples_per_second': 1053.251,
 'eval_steps_per_second': 8.29,
 'epoch': 3.0}

In [103]:
sup_trainer.evaluate(test_dataset)

{'eval_loss': 2.590277671813965,
 'eval_spearman': 0.8161612432361416,
 'eval_runtime': 1.4222,
 'eval_samples_per_second': 1024.491,
 'eval_steps_per_second': 8.438,
 'epoch': 3.0}

---
＜ポイント＞  
・教師なしより教師ありの方が高い性能を示している。  
・対象学習は教師ありの方が良いかも

---

# 最近傍探索ライブラリFaissを使った探索

## 最近傍探索Faiss



---
＜Faiss＞  
・文の集合にSimCSEによる文埋め込みでベクトル化し、それをFaissでインデックスを作成する。  
・クエリのベクトルに類似したベクトルを高速に検索できる。

---

## Faissを利用した最近傍探索の実装

In [104]:
# 流れ
# 教師なしSimSCWでテキストの集合の文埋め込みを取得 -> Faissでインデックスを作成

!pip install datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 4.6 MB/s eta 0:00:0000:0100:01


In [2]:
# wikipediaの段落テキストデータをロードする。
paragraph_dataset = load_dataset('llm-book/jawiki-paragraphs', split="train")

In [3]:
print(paragraph_dataset)
print()

pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。'

In [4]:
# このデータセットは日本語版wikipediaの全記事のほぼすべての段落が含まれている。
# 計算量削減のため、記事の最初の段落のみ使用する
paragraph_datasets = paragraph_dataset.filter(lambda example: example['paragraph_index'] == 0)

In [5]:
print(paragraph_datasets)

print()

pprint(paragraph_datasets[0])
pprint(paragraph_datasets[1])

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 1339236
})

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '10-94194440-0',
 'pageid': 10,
 'paragraph_index': 0,
 'revid': 94194440,
 'section': '__LEAD__',
 'text': '言語(げんご)は、狭義には「声による記号の体系」をいう。',
 'title': '言語'}


In [6]:
# トークナイザとモデルの準備

# 上記で訓練した教師なしSimCSEを読み込む
model_name = "./hidden_files/outputs_unsup_simcse/encoder/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

device = 'cuda:0'
encoder = encoder.to(device)

### モデルによる埋め込み計算

In [7]:
def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""

    # toknizer
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt',
    ).to(device)

    # トークン化されたテキストをベクトル変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(**tokenized_texts).last_hidden_state[:, 0]

    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが１(ベクトルの大きさを1にする)になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    
    return emb    

---
＜ポイント＞  
・SimCSEのエンコーダで入力テキストをベクトルに変換する関数。  
・得られたベクトルはL2正規化を行い、ベクトルの大きさが１になるようにする -> Faissではコサイン類似度が実装されておらず、内積しかしないため  
・推論高速化のため、torch.inference_modeで自動微分を止めて、torch.cuda.amp.autocastで自動混合精度演算を有効にする

---

In [8]:
# 作成したデータのすべての事例に埋め込みをする
paragraph_dataset = paragraph_datasets.map(lambda example: {"embeddings": list(embed_texts(example['text']))}, batched=True)

Parameter 'function'=<function <lambda> at 0x7f76c9139a20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1339236 [00:00<?, ? examples/s]

In [12]:
print(paragraph_datasets[0]['text'])
print(type(paragraph_datasets[0]['text']))

# listじゃなくてstrでも受け取れる
test = tokenizer(paragraph_datasets[0]['text'])
print(test)

アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
<class 'str'>
{'input_ids': [2, 12782, 7184, 16775, 23, 10577, 7779, 12566, 41, 24201, 15865, 7045, 13841, 24, 465, 384, 621, 7196, 1067, 8233, 395, 29, 29, 29, 458, 29, 29, 29, 396, 500, 12831, 12484, 17126, 457, 12485, 385, 15430, 5630, 457, 395, 29, 29, 29, 458, 29, 29, 29, 396, 500, 15329, 14189, 5601, 17, 22719, 17, 464, 1217, 7986, 500, 16706, 458, 12484, 385, 13684, 464, 25403, 457, 484, 384, 30293, 7074, 29640, 12980, 16162, 12496, 12666, 464, 25403, 457, 465, 384, 17, 22719, 17, 464, 1217, 7986, 457, 12485, 12489, 430, 16347, 461, 19597, 1735, 2101, 500, 12586, 441, 456, 12483, 385, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
pprint(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [14]:
pprint(paragraph_dataset[0])

{'embeddings': [0.013611308299005032,
                -0.028840946033596992,
                -0.050424981862306595,
                0.020414190366864204,
                -0.10140922665596008,
                -0.02360052429139614,
                -0.03240034356713295,
                0.040186893194913864,
                0.0370589941740036,
                -0.019628683105111122,
                0.036412011831998825,
                -0.04205688461661339,
                -0.01598992943763733,
                -0.08652804791927338,
                0.033289022743701935,
                0.019001921638846397,
                0.015945274382829666,
                -0.050059735774993896,
                -0.057311445474624634,
                0.022405650466680527,
                0.019865605980157852,
                -0.03222003951668739,
                -0.014888784848153591,
                -0.013414253480732441,
                0.05316145718097687,
                0.010170618072152138,
        

In [15]:
paragraph_dataset.save_to_disk("./hidden_files/outputs_unsup_simcse/embedded_paragraphs")

Saving the dataset (0/10 shards):   0%|          | 0/1339236 [00:00<?, ? examples/s]

### Faissによる最近傍探索をためす

In [41]:
# import faiss

# エンコーダから隠れ層のサイズを取得
emb_dim = encoder.config.hidden_size

# ベクトルの次元数を与えて、空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)

# paragraph_datasetのembeddingsベクトルからFaissインデックスを構築する
# add_faiss_index("インデックスに追加するparagraph_datasetのフィールド名", custom_index:作成したインデックス)-> shapeは(隠れ層の数, paragraph_datasetのデータ数 )
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

  0%|          | 0/1340 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})

In [49]:
query_text = "日本語は、主に日本で話されている言語である。"

# 最近傍探索を実行し、類似度上位10件の事例とスコアを取得する
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(

    # embeddingはparagraph_dataseのフィールド名ではなく、インデックス名
    # embed_texts([query_text])では(1, 768)のshapeなのでembed_texts([query_text])[0]で(768,)で一次元のベクトルにする
    # 上位10の類似度が選抜され、そのインデックスに対応するparagraph_datasetのデータがretriveved_examplesに格納される
    "embeddings", embed_texts([query_text])[0], k=10
)
# 取得した事例の内容をスコアとともに表示する
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]
for score, title, text in zip(scores, titles, texts):
    print(score, title, text)

0.7953488 日本語 日本語(にほんご、にっぽんご、英語: Japanese)は、日本国内や、かつての日本領だった国、そして国外移民や移住者を含む日本人同士の間で使用されている言語。日本は法令によって公用語を規定していないが、法令その他の公用文は全て日本語で記述され、各種法令において日本語を用いることが規定され、学校教育においては「国語」の教科として学習を行う等、事実上、日本国内において唯一の公用語となっている。
0.7873157 日本の言語 日本の言語(にほんのげんご)は、日本の国土で使用されている言語について記述する。日本#言語も参照。
0.76277953 日本語学 日本語学(にほんごがく)とは、日本語を研究の対象とする学問である。
0.74976087 比企 日本語、日本人の姓の一つ。
0.70037663 ジャパン ジャパン(英語: Japan)は、英語で日本を意味する単語。
0.69995856 日本語学校 日本語学校(にほんごがっこう)とは、主に日本語を母語としない者を対象として、第二言語・外国語としての日本語教育を実施する機関。日本国内外に存在している。
0.6929265 日本文学 日本文学(にほんぶんがく)とは、日本語で書かれた文学作品、あるいは日本人が書いた文学、もしくは日本で発表された文学である。中国の古典語である漢文も、日本人によって創作されている場合、日本文学に含まれる。上記の作品やそれらを創作した小説家・詩人などを研究する学問も日本文学と呼ばれる。国文学と呼ばれることもある。
0.6865238 現代日本語文法 現代日本語文法(げんだいにほんごぶんぽう)は、現代(狭義には近代と区別して戦後)の、母語話者によって使われている日本語の文法である。
0.6852733 日本語教育 日本語教育(にほんごきょういく)とは、外国語としての日本語、第二言語としての日本語についての教育の総称である。
0.6657345 日本語放送 日本語放送(にほんごほうそう)とは、広義では日本語による放送全てを指すが、狭義では日本国外からの日本向けの日本語による国際放送を指す。以下では後者について説明する。
